# Datathon 1: Advanced: Travel Insurance Claim Prediction
### Predict if an insurance buyer will claim the insurance or not

Travel insurance is a type of insurance that covers the costs and losses associated with traveling. It is useful protection for those traveling domestically or abroad.

Many companies selling tickets or travel packages, give consumers the option to purchase travel insurance, also known as travelers insurance. Some travel policies cover damage to personal property, rented equipment, such as rental cars, or even the cost of paying a ransom. 

### Problem Statement
Imagine you are working as a data scientist in an insurance company in the USA. The company has collected the data of earlier travel insurance buyers. In this season of vacation, the company wants to know which person will claim their travel insurance and who will not. The company has chosen you to apply your Machine Learning knowledge and provide them with a model that achieves this vision.

### Objective
You are responsible for building a machine learning model for the insurance company to predict if the insurance buyer will claim their travel insurance or not.

### Evaluation Criteria
Submissions are evaluated using F1 Score.

# Load the dataset

In [1]:
# Let's load the data and take a look at it

import pandas as pd

ins_data  = pd.read_csv("https://raw.githubusercontent.com/dphi-official/Datasets/master/travel_insurance/Training_set_label.csv" )
ins_data.head()

,Agency,Agency Type,Distribution Channel,Product Name,Duration,Destination,Net Sales,Commision (in value),Gender,Age,Claim
0,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,61,UNITED KINGDOM,19.8,11.88,NaN,29,0
1,EPX,Travel Agency,Online,Cancellation Plan,93,NEW ZEALAND,63.0,0.00,NaN,36,0
2,EPX,Travel Agency,Online,2 way Comprehensive Plan,22,UNITED STATES,22.0,0.00,NaN,25,0
3,C2B,Airlines,Online,Silver Plan,14,SINGAPORE,54.5,13.63,M,24,0
4,EPX,Travel Agency,Online,Cancellation Plan,90,VIET NAM,10.0,0.00,NaN,23,0


# Exploratory Data Analysis

In [2]:
ins_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48260 entries, 0 to 48259
Data columns (total 11 columns):
Agency                  48260 non-null object
Agency Type             48260 non-null object
Distribution Channel    48260 non-null object
Product Name            48260 non-null object
Duration                48260 non-null int64
Destination             48260 non-null object
Net Sales               48260 non-null float64
Commision (in value)    48260 non-null float64
Gender                  13899 non-null object
Age                     48260 non-null int64
Claim                   48260 non-null int64
dtypes: float64(2), int64(3), object(6)
memory usage: 4.1+ MB


It looks as if we do not have any missing values for 10 columns of our dataset, however there are many missing ones in "Gender" column. We can see how many missing values exist with following code.

In [3]:
ins_data.isnull().sum()

Agency                      0
Agency Type                 0
Distribution Channel        0
Product Name                0
Duration                    0
Destination                 0
Net Sales                   0
Commision (in value)        0
Gender                  34361
Age                         0
Claim                       0
dtype: int64

Gotcha! We do have nulls in the Gender column and looks like it forms a big percentage of the column.

In [4]:
gender_nulls=ins_data.Gender.isnull().sum()/len(ins_data.Gender)

print("{} % of the data is missing for the Gender column.".format(round(gender_nulls*100,3)))

71.2 % of the data is missing for the Gender column.


Let's fill them with "Not Specified" for now because we don't know if the Gender column is an important feature for the model or not.

In [5]:
ins_data["Gender"].fillna("Not Specified",inplace=True)
ins_data.head()

,Agency,Agency Type,Distribution Channel,Product Name,Duration,Destination,Net Sales,Commision (in value),Gender,Age,Claim
0,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,61,UNITED KINGDOM,19.8,11.88,Not Specified,29,0
1,EPX,Travel Agency,Online,Cancellation Plan,93,NEW ZEALAND,63.0,0.00,Not Specified,36,0
2,EPX,Travel Agency,Online,2 way Comprehensive Plan,22,UNITED STATES,22.0,0.00,Not Specified,25,0
3,C2B,Airlines,Online,Silver Plan,14,SINGAPORE,54.5,13.63,M,24,0
4,EPX,Travel Agency,Online,Cancellation Plan,90,VIET NAM,10.0,0.00,Not Specified,23,0


## Pandas Profiling

Now we can move on to descriptive analysis of the dataset.

In [6]:
from pandas_profiling import ProfileReport

profile = ProfileReport(ins_data)
profile


C:\Users\Semanur\Anaconda3\lib\site-packages\pandas_profiling\describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,11
Number of observations,48260
Total Missing (%),0.0%
Total size in memory,4.1 MiB
Average record size in memory,88.0 B
Numeric,4
Categorical,6
Boolean,1
Date,0
Text (Unique),0
Rejected,0


In [7]:
ins_data.describe()

,Duration,Net Sales,Commision (in value),Age,Claim
count,48260.000000,48260.000000,48260.000000,48260.000000,48260.000000
mean,49.546602,40.747647,9.811735,39.927269,0.014671
std,104.828147,48.823217,19.695339,14.027329,0.120231
min,-2.000000,-389.000000,0.000000,0.000000,0.000000
25%,9.000000,18.000000,0.000000,35.000000,0.000000
50%,22.000000,27.000000,0.000000,36.000000,0.000000
75%,53.000000,48.000000,11.630000,43.000000,0.000000
max,4881.000000,666.000000,262.760000,118.000000,1.000000


In [8]:
ins_data.corr()

,Duration,Net Sales,Commision (in value),Age,Claim
Duration,1.000000,0.416221,0.334920,0.002555,0.073068
Net Sales,0.416221,1.000000,0.651033,0.034262,0.137079
Commision (in value),0.334920,0.651033,1.000000,0.118530,0.098935
Age,0.002555,0.034262,0.118530,1.000000,-0.010401
Claim,0.073068,0.137079,0.098935,-0.010401,1.000000


As we can see from the report, we are dealing with a **minority class binary classification** problem. That means, there are 2 classess, namely 0 and 1. But 0's are far more than 1's. Our aim is to predict 1's.

## Duration Column

Another important consideration is the skewness of the Duration column. Minimum duration is negative but time cannot be a negative value. 

In [9]:
ins_data["Duration"].describe()

count    48260.000000
mean        49.546602
std        104.828147
min         -2.000000
25%          9.000000
50%         22.000000
75%         53.000000
max       4881.000000
Name: Duration, dtype: float64

How many entries are negative?

In [10]:
ins_data[ins_data["Duration"]<0]

,Agency,Agency Type,Distribution Channel,Product Name,Duration,Destination,Net Sales,Commision (in value),Gender,Age,Claim
3753,JZI,Airlines,Online,Basic Plan,-1,INDONESIA,18.0,6.3,Not Specified,118,0
10516,JZI,Airlines,Online,Basic Plan,-1,BRUNEI DARUSSALAM,18.0,6.3,Not Specified,118,0
12854,JZI,Airlines,Online,Basic Plan,-2,BANGLADESH,22.0,7.7,M,118,0


In [11]:
# The amount is not big when compared to the length of the dataset. So I chose to remove these.

minus_duration= list(ins_data[ins_data["Duration"]<0].index) # Put the indexes of columns in a list

ins_data.drop(labels=minus_duration, axis=0,inplace=True)
ins_data[ins_data["Duration"]<0]

,Agency,Agency Type,Distribution Channel,Product Name,Duration,Destination,Net Sales,Commision (in value),Gender,Age,Claim


## Destination Column

With the help of pandas crosstab function, we can sum Destinations over Claims. The aim of this is to see if any destination stands out(having more claims).

In [12]:
destinations=pd.crosstab(ins_data["Destination"],ins_data["Claim"])
destinations

Claim,0,1
Destination,,
ALBANIA,1,0
ANGOLA,1,0
ARGENTINA,15,1
ARMENIA,1,0
AUSTRALIA,2776,31
...,...,...
VENEZUELA,5,0
VIET NAM,1289,7
"VIRGIN ISLANDS, U.S.",1,0


Looks like the max number of claims is 426, and average claims for a destination is 5.

In [13]:
destinations.describe()

Claim,0,1
count,141.000000,141.000000
mean,337.226950,5.021277
std,1092.120908,36.179210
min,1.000000,0.000000
25%,2.000000,0.000000
50%,10.000000,0.000000
75%,87.000000,0.000000
max,9642.000000,426.000000


Here are the top 5 columns having most claims.

In [14]:
destinations.sort_values(by=[1],ascending=False)

Claim,0,1
Destination,,
SINGAPORE,9642,426
CHINA,3603,39
AUSTRALIA,2776,31
THAILAND,4480,29
UNITED STATES,1919,25
...,...,...
GUATEMALA,1,0
GUAM,5,0
GUADELOUPE,1,0


## Net Sales - Comission Columns

We expect Net Sales and Comission to be correlated:

In [15]:
ins_data["Net Sales"].corr(ins_data["Commision (in value)"])

0.6510327753918461

Yes, the correlation is not strong but they are correlated.

Net sales cannot be smaller than the commission because most of the time, the comission is represented by a function of net sales.. Let's check if we have such values.

In [16]:
ins_data[ins_data["Net Sales"]<ins_data["Commision (in value)"]]

,Agency,Agency Type,Distribution Channel,Product Name,Duration,Destination,Net Sales,Commision (in value),Gender,Age,Claim
12,EPX,Travel Agency,Online,Cancellation Plan,49,SINGAPORE,-80.0,0.00,Not Specified,21,0
77,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,4,MALAYSIA,0.0,11.88,Not Specified,31,0
138,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,5,AUSTRALIA,0.0,17.82,Not Specified,31,0
169,CCR,Travel Agency,Offline,Comprehensive Plan,34,THAILAND,0.0,9.57,F,118,0
171,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,47,AUSTRALIA,0.0,29.70,Not Specified,34,0
...,...,...,...,...,...,...,...,...,...,...,...
48034,C2B,Airlines,Online,Bronze Plan,19,SINGAPORE,0.0,13.38,M,33,0
48039,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,152,UNITED KINGDOM,0.0,17.82,Not Specified,41,0
48069,EPX,Travel Agency,Online,2 way Comprehensive Plan,50,"TAIWAN, PROVINCE OF CHINA",-50.0,0.00,Not Specified,49,0
48110,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,35,ICELAND,-49.5,29.70,Not Specified,27,0


To fix this, we will fill zero or negative Net Sales with zero comission.

In [17]:
ins_data.loc[ins_data["Net Sales"]<=0]=ins_data["Commision (in value)"].min()

In [18]:
len(ins_data[ins_data["Net Sales"]<ins_data["Commision (in value)"]])

0

# One-Hot Encoding

Now it is time to do one-hot encoding and turn categorical columns into numeric ones. Some of the columns are asked to be dropped as well.

In [19]:
ins_data.drop(['Distribution Channel', 'Destination' ,'Agency Type'],axis=1,inplace=True)
ins_data.head()

,Agency,Product Name,Duration,Net Sales,Commision (in value),Gender,Age,Claim
0,CWT,Rental Vehicle Excess Insurance,61.0,19.8,11.88,Not Specified,29.0,0.0
1,EPX,Cancellation Plan,93.0,63.0,0.00,Not Specified,36.0,0.0
2,EPX,2 way Comprehensive Plan,22.0,22.0,0.00,Not Specified,25.0,0.0
3,C2B,Silver Plan,14.0,54.5,13.63,M,24.0,0.0
4,EPX,Cancellation Plan,90.0,10.0,0.00,Not Specified,23.0,0.0


In [20]:
categoricals=list(ins_data.select_dtypes(include=["object"]).columns)
categoricals

['Agency', 'Product Name', 'Gender']

In [21]:
one_hot=pd.get_dummies(data=ins_data, columns=categoricals)
one_hot.head()

,Duration,Net Sales,Commision (in value),Age,Claim,Agency_0.0,Agency_ADM,Agency_ART,Agency_C2B,Agency_CBH,...,Product Name_Single Trip Travel Protect Platinum,Product Name_Single Trip Travel Protect Silver,Product Name_Spouse or Parents Comprehensive Plan,Product Name_Ticket Protector,Product Name_Travel Cruise Protect,Product Name_Value Plan,Gender_0.0,Gender_F,Gender_M,Gender_Not Specified
0,61.0,19.8,11.88,29.0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,93.0,63.0,0.00,36.0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,22.0,22.0,0.00,25.0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,14.0,54.5,13.63,24.0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,90.0,10.0,0.00,23.0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [22]:
one_hot.columns

Index(['Duration', 'Net Sales', 'Commision (in value)', 'Age', 'Claim',
       'Agency_0.0', 'Agency_ADM', 'Agency_ART', 'Agency_C2B', 'Agency_CBH',
       'Agency_CCR', 'Agency_CSR', 'Agency_CWT', 'Agency_EPX', 'Agency_JWT',
       'Agency_JZI', 'Agency_KML', 'Agency_LWC', 'Agency_RAB', 'Agency_SSI',
       'Agency_TST', 'Agency_TTW', 'Product Name_0.0',
       'Product Name_1 way Comprehensive Plan',
       'Product Name_2 way Comprehensive Plan', 'Product Name_24 Protect',
       'Product Name_Annual Gold Plan', 'Product Name_Annual Silver Plan',
       'Product Name_Annual Travel Protect Gold',
       'Product Name_Annual Travel Protect Platinum',
       'Product Name_Annual Travel Protect Silver', 'Product Name_Basic Plan',
       'Product Name_Bronze Plan', 'Product Name_Cancellation Plan',
       'Product Name_Child Comprehensive Plan',
       'Product Name_Comprehensive Plan', 'Product Name_Gold Plan',
       'Product Name_Individual Comprehensive Plan',
       'Product Name_Pr

In [23]:
one_hot.drop(['Agency_0.0', 'Product Name_0.0', 'Gender_0.0'],axis=1,inplace=True)
one_hot.head()

,Duration,Net Sales,Commision (in value),Age,Claim,Agency_ADM,Agency_ART,Agency_C2B,Agency_CBH,Agency_CCR,...,Product Name_Single Trip Travel Protect Gold,Product Name_Single Trip Travel Protect Platinum,Product Name_Single Trip Travel Protect Silver,Product Name_Spouse or Parents Comprehensive Plan,Product Name_Ticket Protector,Product Name_Travel Cruise Protect,Product Name_Value Plan,Gender_F,Gender_M,Gender_Not Specified
0,61.0,19.8,11.88,29.0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,93.0,63.0,0.00,36.0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,22.0,22.0,0.00,25.0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,14.0,54.5,13.63,24.0,0.0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
4,90.0,10.0,0.00,23.0,0.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


# Handling class imbalance - SMOTE

Let's first seperate our dependent and independent variables.

In [24]:
import numpy as np

X=np.array(one_hot.drop("Claim",axis=1))
X

array([[61.  , 19.8 , 11.88, ...,  0.  ,  0.  ,  1.  ],
       [93.  , 63.  ,  0.  , ...,  0.  ,  0.  ,  1.  ],
       [22.  , 22.  ,  0.  , ...,  0.  ,  0.  ,  1.  ],
       ...,
       [19.  , 29.7 , 17.82, ...,  0.  ,  0.  ,  1.  ],
       [ 7.  , 20.  ,  0.  , ...,  0.  ,  0.  ,  1.  ],
       [35.  , 20.  ,  0.  , ...,  0.  ,  0.  ,  1.  ]])

In [25]:
y=np.array(one_hot["Claim"])
y

array([0., 0., 0., ..., 0., 0., 0.])

**SMOTE** method is used to balance classes. In our case, label 1 in claims are far lower than label 0. This can cause bias in our model and that's why we are usind this library to create synthetic data to equalize classes.

In [26]:
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

print("Number transactions X_train dataset: ", X_train.shape)
print("Number transactions y_train dataset: ", y_train.shape)
print("Number transactions X_test dataset: ", X_test.shape)
print("Number transactions y_test dataset: ", y_test.shape)

Number transactions X_train dataset:  (33779, 48)
Number transactions y_train dataset:  (33779,)
Number transactions X_test dataset:  (14478, 48)
Number transactions y_test dataset:  (14478,)


In [27]:
print("Before OverSampling - # of label 1: {}".format(sum(y_train==1)))
print("Before OverSampling - # of label 0: {} \n".format(sum(y_train==0)))

sm = SMOTE(sampling_strategy=1.0, random_state=25)
X_train_new, y_train_new = sm.fit_sample(X_train, y_train)

print("==============================================")

print('After OverSampling - X_train shape: {}'.format(X_train_new.shape))
print('After OverSampling - t_train shape: {} \n'.format(y_train_new.shape))

print("After OverSampling - # of label 1: {}".format(sum(y_train_new==1)))
print("After OverSampling - # of label 0: {}".format(sum(y_train_new==0)))

Before OverSampling - # of label 1: 491
Before OverSampling - # of label 0: 33288 

After OverSampling - X_train shape: (66576, 48)
After OverSampling - t_train shape: (66576,) 

After OverSampling - # of label 1: 33288
After OverSampling - # of label 0: 33288


# Building the Model

We will try 3 different classification algortihms: Logistic Regression, Decision Tree and Random Forest. Let's start by importing necessary modules.

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score

### Logistic Regression

In [29]:
model = LogisticRegression()
model.fit(X_train_new,y_train_new)
preds=model.predict(X_test)

#print(f1_score(y_test, preds))
#print(accuracy_score(y_test,preds))

# Classification report
print(classification_report(y_test, preds, digits=4))

              precision    recall  f1-score   support

         0.0     0.9939    0.8345    0.9073     14262
         1.0     0.0571    0.6620    0.1052       216

    accuracy                         0.8320     14478
   macro avg     0.5255    0.7483    0.5062     14478
weighted avg     0.9799    0.8320    0.8953     14478



C:\Users\Semanur\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


### Decision Tree

In [30]:
from sklearn.tree import DecisionTreeClassifier

In [31]:
model_DT= DecisionTreeClassifier(random_state=1)
model_DT.fit(X_train,y_train)
pred_DT=model_DT.predict(X_test)
print(f1_score(y_test,pred_DT))
print(accuracy_score(y_test,pred_DT))

0.09375
0.9719574526868352


### Random Forest

In [32]:
from sklearn.ensemble import RandomForestClassifier 

In [33]:
model_rf= RandomForestClassifier(random_state=1)
model_rf.fit(X_train,y_train)
pred_rf=model_rf.predict(X_test)
print(f1_score(y_test,pred_rf))
print(accuracy_score(y_test,pred_rf))

0.03802281368821293
0.9825252106644564


## Grid Search CV - Improve the model

In [34]:
from sklearn.model_selection import GridSearchCV

In [35]:
grid_search1_params = {
    'max_depth': [10, 20],
    'min_samples_leaf': [3, 4],
    'min_samples_split': [4, 6],
    'n_estimators': [100, 200],
             }

grid_search1 = GridSearchCV(model_rf, grid_search1_params, cv=3, verbose=2, n_jobs=-1)
grid_search1.fit(X_train_new, y_train_new)
pred_grid=grid_search1.predict(X_test)

print("==========================================")
print("Best parameters for Grid search is:")
print(grid_search1.best_params_)
print("==========================================")
print(f1_score(y_test,pred_grid))
print(accuracy_score(y_test,pred_grid))

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  2.6min finished


Best parameters for Grid search is:
{'max_depth': 20, 'min_samples_leaf': 3, 'min_samples_split': 4, 'n_estimators': 200}
0.07272727272727272
0.9612515540820555


In [36]:
grid_search2_params = {
    'max_leaf_nodes': [5,50],
    'min_samples_split': [2,8]
             }

grid_search2 = GridSearchCV(model_rf, grid_search2_params, cv=3, verbose=2, n_jobs=-1)
grid_search2.fit(X_train_new, y_train_new)
pred_grid2=grid_search2.predict(X_test)

print("==========================================")
print("Best parameters for Grid search is:")
print(grid_search2.best_params_)
print("==========================================")
print(f1_score(y_test,pred_grid2))
print(accuracy_score(y_test,pred_grid2))

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:   34.9s finished


Best parameters for Grid search is:
{'max_leaf_nodes': 45, 'min_samples_split': 2}
0.12709030100334448
0.8918358889349358


In [37]:
final_model=grid_search2.best_estimator_

In [38]:
final_model

RandomForestClassifier(max_leaf_nodes=45, random_state=1)

## RFE (Recursive Feature Elimination)

RFE is a feature selection algorithm. There are many ways we can evaluate model's features and decide if a column contributes to explanation of our dependent variable. We don't want to add the redundant columns because it slows our model down. Let's see which columns should be included 

In [39]:
from sklearn.feature_selection import RFE

In [40]:
rfe = RFE(final_model,step=1)
rfe.fit(X_train_new,y_train_new)

RFE(estimator=RandomForestClassifier(max_leaf_nodes=45, random_state=1))

In [41]:
for i in range(X_train_new.shape[1]):
    print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))

Column: 0, Selected True, Rank: 1.000
Column: 1, Selected True, Rank: 1.000
Column: 2, Selected True, Rank: 1.000
Column: 3, Selected True, Rank: 1.000
Column: 4, Selected False, Rank: 22.000
Column: 5, Selected False, Rank: 13.000
Column: 6, Selected True, Rank: 1.000
Column: 7, Selected False, Rank: 20.000
Column: 8, Selected False, Rank: 15.000
Column: 9, Selected False, Rank: 16.000
Column: 10, Selected True, Rank: 1.000
Column: 11, Selected True, Rank: 1.000
Column: 12, Selected False, Rank: 2.000
Column: 13, Selected True, Rank: 1.000
Column: 14, Selected False, Rank: 7.000
Column: 15, Selected True, Rank: 1.000
Column: 16, Selected False, Rank: 9.000
Column: 17, Selected False, Rank: 8.000
Column: 18, Selected False, Rank: 3.000
Column: 19, Selected False, Rank: 24.000
Column: 20, Selected True, Rank: 1.000
Column: 21, Selected True, Rank: 1.000
Column: 22, Selected False, Rank: 17.000
Column: 23, Selected False, Rank: 19.000
Column: 24, Selected True, Rank: 1.000
Column: 25, Se

In [42]:
sum=rfe.ranking_[rfe.ranking_==1].sum()

print("Total number of selected features: %d" % (sum))

Total number of selected features: 24


In [43]:
print(f1_score(y_test,rfe.predict(X_test)))
print(accuracy_score(y_test,rfe.predict(X_test)))

0.12794612794612795
0.8926647326978865


In [44]:
print(classification_report(y_test, rfe.predict(X_test), digits=4))

              precision    recall  f1-score   support

         0.0     0.9921    0.8982    0.9428     14262
         1.0     0.0728    0.5278    0.1279       216

    accuracy                         0.8927     14478
   macro avg     0.5324    0.7130    0.5354     14478
weighted avg     0.9784    0.8927    0.9307     14478



In [45]:
test_data = pd.read_csv('https://raw.githubusercontent.com/dphi-official/Datasets/master/travel_insurance/Testing_set_label.csv')
test_data.head()

,Agency,Agency Type,Distribution Channel,Product Name,Duration,Destination,Net Sales,Commision (in value),Gender,Age
0,EPX,Travel Agency,Online,Cancellation Plan,24,HONG KONG,27.0,0.0,NaN,36
1,EPX,Travel Agency,Online,Cancellation Plan,51,JAPAN,45.0,0.0,NaN,36
2,EPX,Travel Agency,Online,Cancellation Plan,52,JAPAN,21.0,0.0,NaN,21
3,EPX,Travel Agency,Online,Cancellation Plan,89,SINGAPORE,11.0,0.0,NaN,30
4,EPX,Travel Agency,Online,Cancellation Plan,5,MALAYSIA,10.0,0.0,NaN,33


In [46]:
test_data.shape

(15832, 10)

In [47]:
test_data["Gender"].fillna("Not Specified",inplace=True)
test_data.head()

,Agency,Agency Type,Distribution Channel,Product Name,Duration,Destination,Net Sales,Commision (in value),Gender,Age
0,EPX,Travel Agency,Online,Cancellation Plan,24,HONG KONG,27.0,0.0,Not Specified,36
1,EPX,Travel Agency,Online,Cancellation Plan,51,JAPAN,45.0,0.0,Not Specified,36
2,EPX,Travel Agency,Online,Cancellation Plan,52,JAPAN,21.0,0.0,Not Specified,21
3,EPX,Travel Agency,Online,Cancellation Plan,89,SINGAPORE,11.0,0.0,Not Specified,30
4,EPX,Travel Agency,Online,Cancellation Plan,5,MALAYSIA,10.0,0.0,Not Specified,33


In [48]:
test_data.drop(['Distribution Channel', 'Destination' ,'Agency Type'],axis=1,inplace=True)
test_data.head()

,Agency,Product Name,Duration,Net Sales,Commision (in value),Gender,Age
0,EPX,Cancellation Plan,24,27.0,0.0,Not Specified,36
1,EPX,Cancellation Plan,51,45.0,0.0,Not Specified,36
2,EPX,Cancellation Plan,52,21.0,0.0,Not Specified,21
3,EPX,Cancellation Plan,89,11.0,0.0,Not Specified,30
4,EPX,Cancellation Plan,5,10.0,0.0,Not Specified,33


In [49]:
test_categoricals=list(test_data.select_dtypes(include=["object"]).columns)
test_categoricals

['Agency', 'Product Name', 'Gender']

In [50]:
test_one_hot=pd.get_dummies(data=test_data, columns=categoricals)
test_one_hot.head()

,Duration,Net Sales,Commision (in value),Age,Agency_ADM,Agency_ART,Agency_C2B,Agency_CBH,Agency_CCR,Agency_CSR,...,Product Name_Single Trip Travel Protect Gold,Product Name_Single Trip Travel Protect Platinum,Product Name_Single Trip Travel Protect Silver,Product Name_Spouse or Parents Comprehensive Plan,Product Name_Ticket Protector,Product Name_Travel Cruise Protect,Product Name_Value Plan,Gender_F,Gender_M,Gender_Not Specified
0,24,27.0,0.0,36,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,51,45.0,0.0,36,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,52,21.0,0.0,21,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,89,11.0,0.0,30,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,5,10.0,0.0,33,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [51]:
rfe.predict(test_one_hot)

array([0., 0., 0., ..., 0., 0., 0.])

In [52]:
res = pd.DataFrame(rfe.predict(test_one_hot)) #preditcions are nothing but the final predictions of your model on input features of your new unseen test data
res.index = test_one_hot.index # its important for comparison. Here "test_new" is your new test dataset
res.columns = ["prediction"]
res.to_csv("prediction_results.csv", index = False)      # the csv file will be saved locally on the same location where this notebook is located.

In [53]:
res

,prediction
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
15827,0.0
15828,1.0
15829,0.0
15830,0.0


In [54]:
res["prediction"].value_counts()

0.0    14093
1.0     1739
Name: prediction, dtype: int64